#### <p style="text-align: center; font-family:cm; font-size:1.8em;">Machine Learning for Dark Matter Signal Classification</p>

<p style="font-family:cm; font-size:1.3em; text-align: center"><b>Owner:</b> Lucas Rocha Castro</p>

<p style="font-family:cm; font-size:1em;"> 
In the process of searching dor a possible Dark Matter (DM) particle, astrophysical observations are one of the pilars of DM indirect detection. When a target is observed, however, it is well known that noise may interfere with the obtained signal, and it is really important to develop strategies in order to be able to separate what is a potential DM particle signal and what is noisy, not usable data. In this project, I stablished a separation system that is simplified, yet physically relevant, taking into account the characteristics of the received signal, such as: the energy received; the angle at which it was intercepted; the particle's velocity and others.
Using this separation, I generated many simulated data points using basic Python functions, and developed a trained algorithm using basic Machine Learning (ML) to evaluate if the machine was able to identify the two type of potential signals or not. 
It is a simple and yet useful project. As an undergraduate in Physics, my main objective is to be able to study ML and other computer science topics while integrating it with my currently researched topics: Dark Matter, Cosmology and Astrophysics.
</p>

---

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import maxwell

#### <p style="text-align: center; font-family:cm; font-size:1.8em;">Data simulation</p>

<p style="font-family:cm; font-size:1em;"> 
I will be following a simplified model to identify what really should be Dark Matter signals versus what is background noise. To <b>Dark Matter</b> signal: <br>
- $E$ (energy) will be a normal distribution with its peak located at ~50GeV, a reasonable value when dealing with indirect detection, such as Gamma Rays. <br>
- $v$ (velocity) will follow a Maxwell-Boltzmann distribution, because according to the Standard Halo Model, the DM particles form an isotropical gas in gravitational equilibrium. Its scale is due to the Sun's velocity around the Galactic Center, which is $v_{\odot} = 220 km/s$. <br>
- $\theta$, as the angle at which those particles are received. Note that it is not isotropic due to the Sun's orbit around the GC, and that causes a type of "dark matter wind", at which there is in fact a prefered angle. <br>
- $\phi$ represents the rotational symmetry due to the Halo's format. <br>
- $r$ represents a DM model that decays exponentially. It is indeed not the current model used (such as NFW, Einasto, Moore, etc.) but in this case, to simplify the analysis and not deal with major computational problems, it is a great approximation. <br>
<br>
In contrast, the majority of <b>background data</b> is distributed on a uniform dataset, which is due to its homogeneity and isotropy, without any peaks because it has no specific events and, furthermore, it is a mix of different processes (radioactivity, cosmic rays, equipment noise and more), which brings a lot of randomness to all parameters.
</p>

In [27]:
#Creating the dataset 
def generate_dm_events(N): 
    E = np.random.normal(loc=50, scale=10, size=N) 
    v = maxwell.rvs(scale=220, size=N) 
    theta = np.random.normal(loc=np.pi/2, scale=0.4, size=N) 
    phi = np.random.uniform(0, 2*np.pi, size=N) 
    r = np.random.exponential(scale=8, size=N) 
    
    return pd.DataFrame({ "E": E, 
                         "v": v, 
                         "theta": theta, 
                         "phi": phi, 
                         "r": r, 
                         "label": 1 })

def generate_noise(N):
    E = np.random.exponential(scale=40, size=N) 
    v = np.random.uniform(0, 600, size=N) 
    theta = np.random.uniform(0, np.pi, size=N) 
    phi = np.random.uniform(0, 2*np.pi, size=N) 
    r = np.random.uniform(0, 20, size=N) 
    
    return pd.DataFrame({ "E": E, 
                         "v": v, 
                         "theta": theta, 
                         "phi": phi, 
                         "r": r, 
                         "label": 0 })

In [57]:
N = 5000
dm = generate_dm_events(N)
bg = generate_noise(N)

data = pd.concat([dm, bg]).sample(frac=1).reset_index(drop=True) 
data.to_csv("events.csv", index=False)

print(data)

              E           v     theta       phi          r  label
0     32.504969  263.554105  1.633818  4.059652   9.502593      1
1     70.505350  440.065908  1.430398  0.198448   2.488079      1
2     76.303426   39.214499  2.159585  5.692631  16.970669      0
3     67.324164  222.361100  1.114934  1.528095   7.310751      1
4     63.967292  229.535305  1.463561  0.741932   1.680408      1
...         ...         ...       ...       ...        ...    ...
9995  86.780548  182.831430  0.526465  2.735062  16.565572      0
9996  47.792378  336.135406  1.295064  1.155807   2.007851      1
9997  21.003633    1.697465  2.792971  6.069804   8.857137      0
9998  68.310034  409.505989  1.515738  3.041167   4.201955      1
9999  52.600210  192.713379  1.665927  2.212616   0.891741      1

[10000 rows x 6 columns]


#### <p style="text-align: center; font-family:cm; font-size:1.8em;">Machine Learning</p>


In [58]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

<p style="font-family:cm; font-size:1.4em;">Logistic Regression</p>
<br>
<p style="font-family:cm; font-size:1em;"> 
Logistic Regression is a ML model that is usually applied to linear data, in which every parameter is associated to a coefficient. It then plots a hyperplane in the 4d space formed by the 4 parameters in question. If a data point falls into one side of the hyperplane, it will be considered Dark Matter. If not, then it will be counted as background noise. <br>
<br>
At the end, the results are compared to the original labels and the accuracy is computed.
</p>

In [68]:
blind_data = data.drop('label', axis = 1) #Separating labels from the data
answer = data["label"] #Answers: 0 for background, 1 for DM

#Separating train and test data
X_train, X_test, y_train, y_test = train_test_split(blind_data, 
                                                    answer, 
                                                    test_size=0.2, 
                                                    random_state=13)
#Putting all into a same scale to avoid absolute value bias
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

#Applying Logistic Regression to the standardized dataset
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

#Computing the final accuracy of DM-signal separation
accuracy = model.score(X_test_scaled, y_test)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.6385


<p style="font-family:cm; font-size:1em;">
The result shows that it is, in fact, very difficult to distinguish between potential dark matter signals and background noise. If results were extremely precise, there would be a chance that the dataset is misleading, or rather the test data could be leaking into training data. This percentage is great for the scientific difficulty of the problem.
</p>
